In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
import time

In [2]:
# Parametri dell'ambiente
hp_reset_noise_scale = 0.036384281716755174  # Scala del rumore quando l'ambiente viene resettato
hp_forward_reward_weight = 1.6734584377802377  # Peso del reward per il movimento in avanti
hp_ctrl_cost_weight = 1.4114977503409765  # Peso del reward per il controllo
hp_healthy_reward = 2.3778485263135485  # Reward per la salute

hp_contact_cost_weight = 5.2035838720379083e-05  # Peso del costo di contatto
healthy_z = (0.28072846666427437, 1.0352095932018308)  # Intervallo di altezza considerato sano
contact_force = (-1.1644341511298375, 1.0234607887152494)  # Intervallo di forza di contatto

In [3]:
env_render = gym.make(
    "Ant-v5",
    reset_noise_scale=hp_reset_noise_scale,
    forward_reward_weight=hp_forward_reward_weight,
    ctrl_cost_weight=hp_ctrl_cost_weight,
    healthy_reward=hp_healthy_reward,
    contact_cost_weight=hp_contact_cost_weight,
    healthy_z_range=healthy_z,
    contact_force_range=contact_force,
    render_mode='human'
)
"""
Questo snippet di codice inizializza un ambiente Gym per il compito "Ant-v5" con specifici iperparametri.

Parametri:
- reset_noise_scale (float): Scala del rumore aggiunto allo stato iniziale durante il reset dell'ambiente.
- forward_reward_weight (float): Peso della ricompensa per il movimento in avanti.
- ctrl_cost_weight (float): Peso del costo associato alle azioni di controllo.
- healthy_reward (float): Ricompensa per mantenere uno stato di salute.
- contact_cost_weight (float): Peso del costo associato alle forze di contatto.
- healthy_z_range (tuple): Intervallo di valori z considerati sani per l'agente.
- contact_force_range (tuple): Intervallo di forze di contatto considerate accettabili.
- render_mode (str): Modalità di rendering dell'ambiente, impostata su 'human' per il rendering visivo.

Questa configurazione è tipicamente utilizzata negli esperimenti di apprendimento per rinforzo per addestrare e valutare agenti nell'ambiente "Ant-v5".
"""

In [ ]:
env_render.training = False # Setta l'environment in modalità di valutazione
env_render.norm_reward = False # Disabilita la normalizzazione della reward. Questo è importante per valutare correttamente il modello.

: 

In [ ]:
# Carica il modello salvato
model = PPO.load("ppo_Ant_model_PPO18")

# Reset dell'ambiente e inizializzazione delle variabili
obs, _ = env_render.reset()
done = False

# Loop di esecuzione fino a quando l'episodio non è terminato
while not done:
    # Predice l'azione basata sull'osservazione corrente
    action, _ = model.predict(obs)
    
    # Esegue l'azione nell'ambiente e ottiene la nuova osservazione e lo stato di completamento
    obs, _, done, _, _ = env_render.step(action)
    
    # Renderizza l'ambiente
    env_render.render()
    
    # Pausa per rendere il rendering visibile
    time.sleep(0.01)

# Chiude l'ambiente
env_render.close()

2025-02-21 21:07:09.747 Python[9039:415726] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-21 21:07:09.747 Python[9039:415726] +[IMKInputSession subclass]: chose IMKInputSession_Modern
/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)
